<a href="https://colab.research.google.com/github/AmineMassaabi/CryptoForecast/blob/master/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import time
import pymongo
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
from statistics import stdev 
import json

myclient = pymongo.MongoClient(host = '139.162.167.213:27017',username="mongo-naascap",password="n44$c4p")
cw = "?api_key=XRSFO6KHEQC87X8UOYZJ"

ts='6h'
market='coinbase-pro'
quote='btc'



def boxAsset(t,c):
    cbp=myclient["Spot"]
    bn=cbp.collection_names()
    l=[word.split("_")[2] for word in bn if(word.startswith(t+"_"+c))]
    l.sort()
    return l

def getAssetClose(t,c,a):
    cbp=myclient["Spot"]
    db=cbp[t+"_"+c+"_"+a]
    db=pd.DataFrame(list(db.find({},{"_id":1,"close":1}).sort("_id",-1)))
    db.rename(columns={'close':a},inplace=True)
    db.index=[(datetime.fromtimestamp(i)) for i in db._id]
    return db[a]


def getMarketCloses(ts):
    assets=boxAsset(ts,market)
    closes=[getAssetClose(ts,market,asset) for asset in assets]
    data = pd.concat(closes,join='outer',axis=1,sort=False)
    date_format = "%d/%m/%Y"
    date = datetime.strptime('01/01/2020', date_format)
    data=data[[date<=dates for dates in data.index]]
    return data


def getColumns(TT):
    df=pd.DataFrame([TT.columns,TT.columns]).T.set_axis(['id', 'name'], axis=1, inplace=False)
    L=[0]*(len(df))
    for i in (range(len(df))):
        L[i]={'id':df["id"][i],'name':df["name"][i]}
    return L
 
def listToDropDown(l):
    df=pd.DataFrame([l,l]).T.set_axis(['Value', 'Label'], axis=1, inplace=False)
    df=df.sort_values("Value").reset_index()
    L=[0]*len(df)
    for i in range(len(df)):
        L[i]={'label':df["Label"][i],'value':df["Value"][i]}
    return L




def TableBaseFromSymbol():
    URL="https://api.cryptowat.ch/pairs"
    res = requests.get(URL + cw)
    T=json_normalize(json.loads(res.content.decode('utf-8'))["result"])
    T=T.loc[:,["symbol","base.symbol","quote.symbol"]]
    return T
T = TableBaseFromSymbol()
def BaseFromSymbol(T,sym):
    if(sym in list(T.symbol)):
        return T["base.symbol"][T.symbol==sym].iloc[0]

def QuoteFromSymbol(T,sym):
    if(sym in list(T.symbol)):
        return T["quote.symbol"][T.symbol==sym].iloc[0]

def getAllAssetCloseByTime(t):
    cbp=myclient["Spot"]
    bn=cbp.collection_names()
    bn=[word for word in bn if(word.split("_")[1]== market and word.split("_")[0]==t)]
    df=pd.DataFrame()
    l2=[]
    l=list(set([word.split("_")[2] for word in bn]))
    TT=list(set([BaseFromSymbol(T,i) for i in l if(QuoteFromSymbol(T,i)) == quote]))
    for asset in TT:
        l1=[word for word in bn if(BaseFromSymbol(T,word.split("_")[2])==asset and QuoteFromSymbol(T,word.split("_")[2]) == quote)]
        rr=[pd.DataFrame(list(cbp[i].find({},{ "_id": 1, "close": 1})),index=pd.DataFrame(list(cbp[i].find({},{ "_id": 1})))._id) for i in l1]
        data = pd.concat([i["close"] for i in rr],join='outer',axis=1,sort=False)
        l2.append(data.mean(axis=1))

    df=pd.concat(l2,axis=1)
    df.columns=(TT)
    df.index=[(datetime.fromtimestamp(i)) for i in df.index]
    df = df.fillna(method = "ffill")
    
    return df

def getAllAssetVolumeByTime(t):
    cbp=myclient["Spot"]
    bn=cbp.collection_names()
    bn=[word for word in bn if(word.split("_")[1]== market and word.split("_")[0]==t)]
    df=pd.DataFrame()
    l2=[]
    l=list(set([word.split("_")[2] for word in bn]))
    TT=list(set([BaseFromSymbol(T,i) for i in l if(QuoteFromSymbol(T,i)) == quote]))
    for asset in TT:
        l1=[word for word in bn if(BaseFromSymbol(T,word.split("_")[2])==asset and QuoteFromSymbol(T,word.split("_")[2]) == quote)]
        rr=[pd.DataFrame(list(cbp[i].find({},{ "_id": 1, "volume_quote": 1})),index=pd.DataFrame(list(cbp[i].find({},{ "_id": 1})))._id) for i in l1]
        data = pd.concat([i["volume_quote"] for i in rr],join='outer',axis=1,sort=False)
        l2.append(data.mean(axis=1))

    df=pd.concat(l2,axis=1)
    df.columns=(TT)
    df.index=[(datetime.fromtimestamp(i)) for i in df.index]
    df = df.fillna(0)
    
    return df


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [ ]:
market='coinbase-pro' #'kraken'
quote='usd' #'btc'
ts='1d' # '5m' '2h' '6h' '12h' '1d'
df=getAllAssetCloseByTime(ts)
df1=getAllAssetVolumeByTime('1d') 

In [15]:
df=df.iloc[1435:,:]

In [16]:
df1=df1.iloc[1435:,:]

In [17]:
df

,band,mkr,xtz,atom,rep,zrx,nmr,etc,oxt,knc,eth,ltc,xlm,dai,eos,dash,bch,link,algo,comp,omg,xrp,btc
2019-01-01,NaN,NaN,NaN,NaN,NaN,0.292672,NaN,4.970,NaN,NaN,130.67,29.78,NaN,NaN,NaN,NaN,147.64,NaN,NaN,NaN,NaN,NaN,3691.86
2019-01-02,NaN,NaN,NaN,NaN,NaN,0.317406,NaN,5.260,NaN,NaN,139.75,31.68,NaN,NaN,NaN,NaN,163.66,NaN,NaN,NaN,NaN,NaN,3826.10
2019-01-03,NaN,NaN,NaN,NaN,NaN,0.325100,NaN,5.350,NaN,NaN,153.50,32.93,NaN,NaN,NaN,NaN,170.00,NaN,NaN,NaN,NaN,NaN,3890.79
2019-01-04,NaN,NaN,NaN,NaN,NaN,0.317246,NaN,5.030,NaN,NaN,147.23,31.60,NaN,NaN,NaN,NaN,159.88,NaN,NaN,NaN,NaN,NaN,3787.57
2019-01-05,NaN,NaN,NaN,NaN,NaN,0.319433,NaN,5.220,NaN,NaN,153.24,32.02,NaN,NaN,NaN,NaN,159.39,NaN,NaN,NaN,NaN,NaN,3820.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-17,14.7365,694.0055,4.1782,6.028,21.53,0.489691,NaN,7.230,0.4379,1.8457,428.89,62.09,0.106117,1.010306,3.800,94.760,310.89,19.02135,0.6471,201.69,1.9132,0.3008,11873.28
2020-08-18,15.0215,722.7416,4.2342,6.233,22.10,0.578261,NaN,7.186,0.7242,1.9153,424.45,63.19,0.112778,1.007524,3.823,94.703,306.08,19.28904,0.6718,195.50,2.9500,0.3006,11821.90
2020-08-19,13.7698,666.4729,4.0552,6.023,21.37,0.579349,48.0712,7.639,0.5926,1.7609,429.18,66.46,0.114549,1.004567,3.733,96.868,318.69,16.80738,0.5762,184.62,2.7158,0.3165,12277.41
2020-08-20,11.8851,643.7986,3.6952,5.454,20.14,0.516479,42.8708,7.091,0.5211,1.5642,411.60,62.68,0.103631,1.011992,3.408,89.722,296.15,15.42238,0.5302,189.20,2.7488,0.2912,11751.14
